## Note on 4/15/2022: Attempt to clean cities names using standardized zip and city name. Alteryx has process doing this

Load Packages

In [1]:
# pip install fuzzywuzzy

In [2]:
# pip install python-Levenshtein

In [3]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

## Proces zip codes

In [4]:
##https://www.unitedstateszipcodes.org/zip-code-database/

In [5]:
cityzip_tb = pd.read_csv('data/zip_code_database.csv')
cityzip_tb = cityzip_tb.rename(columns=str.lower)
cityzip_tb = cityzip_tb.apply(lambda x: x.astype(str).str.upper())

In [6]:
cityzip_tb.columns

Index(['zip', 'type', 'decommissioned', 'primary_city', 'acceptable_cities',
       'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes',
       'world_region', 'country', 'latitude', 'longitude',
       'irs_estimated_population'],
      dtype='object')

In [7]:
cityzip_tb = cityzip_tb[['zip', 'primary_city', 'acceptable_cities', 'state']]

In [8]:
## if zip length is less than 5 then patch leading 0. If length is > 5 and < 9, then patch zero to 9 positions
cityzip_tb['zip_5d'] = cityzip_tb['zip'].apply(lambda x: x.zfill(5) 
                                               if len(x) < 5 
                                               else (x.zfill(9)[0:5] if len(x) < 9 and len(x) > 5 else x[0:5]))

In [9]:
cityzip_tb['acceptable_cities'] = cityzip_tb['acceptable_cities'].replace('NAN',np.nan)

In [10]:
cityzip_tb = cityzip_tb[['zip', 'zip_5d', 'primary_city', 'acceptable_cities']]

In [11]:
cityzip_tb

,zip,zip_5d,primary_city,acceptable_cities
0,501,00501,HOLTSVILLE,NaN
1,544,00544,HOLTSVILLE,NaN
2,601,00601,ADJUNTAS,NaN
3,602,00602,AGUADA,NaN
4,603,00603,AGUADILLA,RAMEY
...,...,...,...,...
42719,99926,99926,METLAKATLA,NaN
42720,99927,99927,POINT BAKER,NaN
42721,99928,99928,WARD COVE,NaN
42722,99929,99929,WRANGELL,NaN


In [12]:
acceptable_cities = cityzip_tb['acceptable_cities'].str.split(',', expand=True)
acceptable_cities = acceptable_cities.replace({'':np.nan, None:np.nan})
acceptable_cities = pd.concat([acceptable_cities, cityzip_tb[['zip_5d', 'primary_city']]], axis = 1)
print(acceptable_cities)

              0        1    2    3    4    5    6    7    8    9  ...   23  \
0           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
1           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
2           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
3           NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
4         RAMEY      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
...         ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
42719       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
42720       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
42721       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
42722       NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
42723  EDNA BAY   KASAAN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   

        24   25   26   27   28   29   30 zip_5d primary_city  


In [13]:
cleaned_cities = pd.melt(acceptable_cities, id_vars =['zip_5d','primary_city'])[['zip_5d','primary_city', 'value']]

In [14]:
cleaned_cities = cleaned_cities.rename(columns={"value": "acceptable_cities"})

In [15]:
cleaned_cities = pd.melt(cleaned_cities, id_vars =['zip_5d'])

In [16]:
cleaned_cities

,zip_5d,variable,value
0,00501,primary_city,HOLTSVILLE
1,00544,primary_city,HOLTSVILLE
2,00601,primary_city,ADJUNTAS
3,00602,primary_city,AGUADA
4,00603,primary_city,AGUADILLA
...,...,...,...
2648883,99926,acceptable_cities,NaN
2648884,99927,acceptable_cities,NaN
2648885,99928,acceptable_cities,NaN
2648886,99929,acceptable_cities,NaN


In [17]:
cleaned_cities = cleaned_cities.dropna().drop_duplicates()

In [18]:
cleaned_cities = cleaned_cities.rename(columns={'value': 'primary_city'})
print(cleaned_cities)

        zip_5d           variable primary_city
0        00501       primary_city   HOLTSVILLE
1        00544       primary_city   HOLTSVILLE
2        00601       primary_city     ADJUNTAS
3        00602       primary_city       AGUADA
4        00603       primary_city    AGUADILLA
...        ...                ...          ...
2453418  41465  acceptable_cities        SEITZ
2496142  41465  acceptable_cities       STELLA
2538866  41465  acceptable_cities      SUBLETT
2581590  41465  acceptable_cities     SWAMPTON
2624314  41465  acceptable_cities       WONNIE

[57611 rows x 3 columns]


In [19]:
cleaned_cities_list = list(cleaned_cities['primary_city'].drop_duplicates())

## Process Grantee table

In [20]:
grantee_tb = pd.read_csv('data/2020_990_python.csv')
grantee_tb = grantee_tb.rename(columns=str.lower)
grantee_tb = grantee_tb.apply(lambda x: x.astype(str).str.upper())

In [21]:
grantee_tb = grantee_tb[['ein', 'grantee', 'address', 'city', 'state', 'zip']]

In [22]:
## if zip length is less than 5 then patch leading 0. If length is > 5 and < 9, then patch zero to 9 positions
grantee_tb['zip_5d'] = grantee_tb['zip'].apply(lambda x: x.zfill(5) 
                                               if len(x) < 5 
                                               else (x.zfill(9)[0:5] if len(x) < 9 and len(x) > 5 else x[0:5]))

In [23]:
grantee_tb = grantee_tb.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [24]:
grantee_tb

,ein,grantee,address,city,state,zip,zip_5d
0,736291151,PHILBROOK MUSEUM OF ART,2727 ROCKFORD ROAD,TULSA,OK,74114,74114
1,736291151,FRED JONES JR MUSEUM OF ART AT U OF,555 ELM AVENUE,NORMAN,OK,73019,73019
2,810714187,DIOCESAN DEVELOPMENT,27 C ST,SALT LAKE CITY,UT,84103,84103
3,810714187,CATHEDRAL OF THE MADELEINE SLC,331 E SOUTH TEMPLE,SALT LAKE CITY,UT,84111,84111
4,810714187,ST JOSEPH'S CATHOLIC ELEMENTARY SCH,1790 LAKE ST,OGDEN,UT,84401,84401
...,...,...,...,...,...,...,...
95667,66422706,FIRST UNITED METHODIST CHURCH,400 BISCAYNE BLVD,MIAMI,FL,33132,33132
95668,66422706,CHILDRENS HEALTHCARE OF ATLANTA FDN,PARK NORTH 1577 NE EXPY STE A,ATLANTA,GA,30329,30329
95669,66422706,GEORGIA INSTITUTE OF TECHNOLOGY FOU,760 SPRING STREET NW - SUITE 400,ATLANTA,GA,30308,30308
95670,66422706,EPWORTH VILLAGE,5300 WEST 16TH AVE,HIALEAH,FL,33012,33012


In [25]:
grantee_tb.loc[grantee_tb['ein'] == '134266189']

,ein,grantee,address,city,state,zip,zip_5d
1136,134266189,PENN STATE RULE PROGRAM,110 TECHNOLOGY CENTER BUILDING,UNIVERSITY PARK,PA,168027000,16802
1137,134266189,PA FRIENDS OF AGRICULTURE FOUND,PO BOX 8736,CAMP HILL,PA,170018736,17001
1138,134266189,LEAD NY,CORNELL UNIVERSITY 275B WARREN HALL,ITHACA,NY,14853,14853
1139,134266189,NYS JR DAIRY LEADER,CORNELL UNIV 272 MORRISON HALL,ITHACA,NY,14853,14853
1140,134266189,NA INTERCOLLEGIATE DAIRY CHALLENGE,ONE TECHNOLOGY PLACE,HOMER,NY,13077,13077
1141,134266189,ST LAWRENCE-LEWIS BOCES,40 WEST MAIN STREET,CANTON,NY,13617,13617
1142,134266189,GENEVA SCHOOL DISTRICT,400 WEST NORTH STREET,GENEVA,NY,14456,14456
1143,134266189,CTR FOR DAIRY EXCELLENCE FDTN PA,ATTN MIRIAM KELLY 2310 N CAMERON,HARRISBURG,PA,17110,17110
1144,134266189,HOLSTEIN FOUNDATION - YDLI,1 HOSLSTEIN PLACE PO BOX 816,BRATTLEBORO,VT,53020816,05302
1145,134266189,PA FFA,PO BOX 157,EAST BERLIN,PA,17316,17316


In [26]:
grantee_tb.loc[grantee_tb['ein'] == '814808588']

,ein,grantee,address,city,state,zip,zip_5d
113,814808588,INFANT CRISIS SERVICES,4224 N LINCOLN BLVD,OKLAHOMA CITY,OK,73105,73105
114,814808588,THE LEUKEMIA LYMPHOMA SOCIETY,3 INTERNATIONAL DR,RYE BROOK,NY,10573,10573
115,814808588,REGIONAL FOOD BANK,PO BOX 270968,OKLAHOMA CITY,OK,731370968,73137
116,814808588,TENACIOUSLY TEAL,720 W WILSHIRE BLVD,OKLAHOMA CITY,OK,73116,73116
117,814808588,KIPP OKC,PO BOX 776,OKLAHOMA CITY,OK,73101,73101
118,814808588,BOYS AND GIRLS CLUB,PO BOX 18701,OKLAHOMA CITY,OK,73154,73154
119,814808588,MAKE-A-WISH,1900 NW EXPRESSWAY,OKLAHOMA CITY,OK,73118,73118
120,814808588,DEADCENTER FILM FESTIVAL,701 W SHERIDAN AVE,OKLAHOMA CITY,OK,73102,73102
121,814808588,ST JUDE CHILDREN'S RESEARCH HOSPIT,400 N WALKER AVE,OKLAHOMA CITY,OK,73102,73102
122,814808588,FIRST DESCENTS,3001 BRIGHTON BLVD,DENVER,CO,80216,80216


In [27]:
df = grantee_tb

In [28]:
# df = pd.merge(left = grantee_tb, how = 'left', right = cityzip_tb
#                      , left_on=['zip_5d'], right_on=['zip_5d'])

In [29]:
df.count()

ein        95672
grantee    95672
address    95672
city       95672
state      95672
zip        95672
zip_5d     95672
dtype: int64

In [30]:
df.head(3)

,ein,grantee,address,city,state,zip,zip_5d
0,736291151,PHILBROOK MUSEUM OF ART,2727 ROCKFORD ROAD,TULSA,OK,74114,74114
1,736291151,FRED JONES JR MUSEUM OF ART AT U OF,555 ELM AVENUE,NORMAN,OK,73019,73019
2,810714187,DIOCESAN DEVELOPMENT,27 C ST,SALT LAKE CITY,UT,84103,84103


In [31]:
df['match_flag'] = df.apply(lambda x: 1 if x['city'] in cleaned_cities_list else 0, axis=1)

In [32]:
df.head(5)

,ein,grantee,address,city,state,zip,zip_5d,match_flag
0,736291151,PHILBROOK MUSEUM OF ART,2727 ROCKFORD ROAD,TULSA,OK,74114,74114,1
1,736291151,FRED JONES JR MUSEUM OF ART AT U OF,555 ELM AVENUE,NORMAN,OK,73019,73019,1
2,810714187,DIOCESAN DEVELOPMENT,27 C ST,SALT LAKE CITY,UT,84103,84103,1
3,810714187,CATHEDRAL OF THE MADELEINE SLC,331 E SOUTH TEMPLE,SALT LAKE CITY,UT,84111,84111,1
4,810714187,ST JOSEPH'S CATHOLIC ELEMENTARY SCH,1790 LAKE ST,OGDEN,UT,84401,84401,1


In [33]:
df.count()

ein           95672
grantee       95672
address       95672
city          95672
state         95672
zip           95672
zip_5d        95672
match_flag    95672
dtype: int64

## Total mismatch

In [34]:
mismatch_df = df.loc[df['match_flag'] == 0]

In [35]:
mismatch_df['match_flag'].count()

3437

In [36]:
mismatch_df = mismatch_df.loc[(mismatch_df['city'].str.contains('VARIOUS|ATTACH') == False)]
mismatch_df['match_flag'].count()

3174

In [37]:
primary_cities = cleaned_cities.loc[cleaned_cities['variable'] == 'primary_city'][['zip_5d', 'primary_city']]

In [38]:
mismatch_df = pd.merge(left = mismatch_df, how = 'left', right = primary_cities
                     , left_on=['zip_5d'], right_on=['zip_5d'])

In [39]:
mismatch_df['primary_city'] = mismatch_df['primary_city'].fillna('######')

In [40]:
mismatch_df

,ein,grantee,address,city,state,zip,zip_5d,match_flag,primary_city
0,770454276,RIO DEL MAR IMPROVEMENT ASSOCIATION,P O BOX 274,RIO DEL MAR,CA,95001,95001,0,APTOS
1,364191712,US HOLOCAUST MEMORIAL MUSEUM,100 RAOUL WALLENBERG PLACE,SW WASHINGTON,DC,20024,20024,0,WASHINGTON
2,66051671,YELLOWSTONE FOREVER,PO BOX 117,YELLOWSTONE NATIONAL,WY,82190,82190,0,YELLOWSTONE NATIONAL PARK
3,954609975,HELEN WOODWARD ANIMALS,6461 EL APAJO RD,RANCHO SANTE FE,CA,92067,92067,0,RANCHO SANTA FE
4,912006733,WASHINGTON STATE ANIMAL RESPONSE TE,PO BOX 21,UNUMCLAW,WA,98022,98022,0,ENUMCLAW
...,...,...,...,...,...,...,...,...,...
3169,262409879,MORGAN STEMPLE,2689 NE HWY 351,VCROSS CITY,FL,32680,32680,0,OLD TOWN
3170,203957959,LIGHT MINISTERIES,PO BOX 328,ROUGE RIVER,OR,97537,97537,0,ROGUE RIVER
3171,208114127,SPRINGS OF LIFE CHURCH,9000 NYE ROAD,CASITAS SPRINGS,CA,93001,93001,0,VENTURA
3172,208114127,PRECEPTS MINISTRIES,7324 NOAH REID ROAD,CHATANOOGA,TN,37421,37421,0,CHATTANOOGA


In [41]:
def checker(x, correct_options, threshold):
    names_array=[]
    ratio_array=[]
    if x['match_flag'] == 1:
        x['suggest_city'] = x['primary_city']
        x['score'] = np.nan
        return x
    
    correct_options = [x['primary_city']]
#     result = process.extractOne(x['city'],correct_options,scorer=fuzz.token_set_ratio)
    result = process.extractOne(x['city'],correct_options,scorer=fuzz.token_set_ratio)
    
    if result[1] > threshold:
        x['suggest_city'] = result[0]
        x['score'] = result[1]
    else:
        x['suggest_city'] = np.nan
        x['score'] = result[1]
    return x

In [42]:
# str2Match = mismatch_df['city'].fillna('######').tolist()

In [43]:
process.extract("SW WASHINGTON", ['WASHINGTON'], scorer = fuzz.ratio)

[('WASHINGTON', 87)]

In [44]:
process.extract("SW WASHINGTON", ['WASHINGTON'], scorer = fuzz.partial_ratio)

[('WASHINGTON', 100)]

In [45]:
process.extract("SW WASHINGTON", ['WASHINGTON'], scorer = fuzz.token_set_ratio)

[('WASHINGTON', 100)]

In [46]:
threshold = 80
def apply_checker(x): return checker(x, cleaned_cities_list, threshold)

In [47]:
mismatch_df = mismatch_df.apply(apply_checker, axis=1)

In [48]:
mismatch_df2 = mismatch_df.loc[mismatch_df['suggest_city'].isnull() == False]
print(mismatch_df2)

            ein                              grantee  \
1     364191712         US HOLOCAUST MEMORIAL MUSEUM   
2      66051671                  YELLOWSTONE FOREVER   
3     954609975               HELEN WOODWARD ANIMALS   
4     912006733  WASHINGTON STATE ANIMAL RESPONSE TE   
5     453982905                      POLARIS PROJECT   
...         ...                                  ...   
3167  364316097         ST LUKE FOUNDATION FOR HAITI   
3168  656237736              AMERICAN CANCER SOCIETY   
3170  203957959                    LIGHT MINISTERIES   
3172  208114127                  PRECEPTS MINISTRIES   
3173  943345871                  SHRINER'S HOSPITALS   

                         address                  city state    zip zip_5d  \
1     100 RAOUL WALLENBERG PLACE         SW WASHINGTON    DC  20024  20024   
2                     PO BOX 117  YELLOWSTONE NATIONAL    WY  82190  82190   
3               6461 EL APAJO RD       RANCHO SANTE FE    CA  92067  92067   
4              

## Data set cleaned by fuzzy matching

In [49]:
mismatch_df.loc[mismatch_df['suggest_city'].isnull() == False]['suggest_city'].count()

1926

In [50]:
mismatch_df.loc[mismatch_df['suggest_city'].isnull() == False]

,ein,grantee,address,city,state,zip,zip_5d,match_flag,primary_city,suggest_city,score
1,364191712,US HOLOCAUST MEMORIAL MUSEUM,100 RAOUL WALLENBERG PLACE,SW WASHINGTON,DC,20024,20024,0,WASHINGTON,WASHINGTON,100
2,66051671,YELLOWSTONE FOREVER,PO BOX 117,YELLOWSTONE NATIONAL,WY,82190,82190,0,YELLOWSTONE NATIONAL PARK,YELLOWSTONE NATIONAL PARK,100
3,954609975,HELEN WOODWARD ANIMALS,6461 EL APAJO RD,RANCHO SANTE FE,CA,92067,92067,0,RANCHO SANTA FE,RANCHO SANTA FE,93
4,912006733,WASHINGTON STATE ANIMAL RESPONSE TE,PO BOX 21,UNUMCLAW,WA,98022,98022,0,ENUMCLAW,ENUMCLAW,88
5,453982905,POLARIS PROJECT,PO BOX 65323,WASHINGOTN,DC,20035,20035,0,WASHINGTON,WASHINGTON,90
...,...,...,...,...,...,...,...,...,...,...,...
3167,364316097,ST LUKE FOUNDATION FOR HAITI,58 LYNN ROAD,IVORYTOWN,CT,6442,06442,0,IVORYTON,IVORYTON,94
3168,656237736,AMERICAN CANCER SOCIETY,8364 HICKMAN RD,DESMOINE,IA,50310,50310,0,DES MOINES,DES MOINES,89
3170,203957959,LIGHT MINISTERIES,PO BOX 328,ROUGE RIVER,OR,97537,97537,0,ROGUE RIVER,ROGUE RIVER,91
3172,208114127,PRECEPTS MINISTRIES,7324 NOAH REID ROAD,CHATANOOGA,TN,37421,37421,0,CHATTANOOGA,CHATTANOOGA,95


## Remaining not matching

In [51]:
mismatch_df.loc[mismatch_df['suggest_city'].isnull() == True].count()

ein             1248
grantee         1248
address         1248
city            1248
state           1248
zip             1248
zip_5d          1248
match_flag      1248
primary_city    1248
suggest_city       0
score           1248
dtype: int64

In [52]:
mismatch_df.loc[mismatch_df['suggest_city'].isnull() == True]

,ein,grantee,address,city,state,zip,zip_5d,match_flag,primary_city,suggest_city,score
0,770454276,RIO DEL MAR IMPROVEMENT ASSOCIATION,P O BOX 274,RIO DEL MAR,CA,95001,95001,0,APTOS,NaN,25
7,954664892,OPERATION INTEGRITY,24040 CAMINO DEL AVION A115,MONARCH BEACH,CA,92629,92629,0,DANA POINT,NaN,35
9,831602675,JEFF LICHTERMAN,2101 COVE ROAD,TEQUESTA,FL,33469,33469,0,JUPITER,NaN,27
13,810945657,OTHER MISC DONATIONS,BROOKLYN,NY,NY,11219,11219,0,BROOKLYN,NaN,20
15,236417708,DENISE PINTO,60 HOSPITAL STREET,JENKINS TWP,PA,18640,18640,0,PITTSTON,NaN,32
...,...,...,...,...,...,...,...,...,...,...,...
3137,465588406,KYLE NICOLLE PELTIER,8011 PINEWOOD DRIVE,HAM LAKE,MN,55304,55304,0,ANDOVER,NaN,27
3140,311483132,LOXLEY CHASE,3736 KINGS ROW,WINSTONSALEM,NC,27106,27106,0,WINSTON SALEM,NaN,56
3165,263899785,THE MYELCDYSPLASTC SYNDROMES FOUNDA,4573 SOUTH BROAD STREET SUITE 150,YARDVILLE,NJ,8620,08620,0,TRENTON,NaN,25
3169,262409879,MORGAN STEMPLE,2689 NE HWY 351,VCROSS CITY,FL,32680,32680,0,OLD TOWN,NaN,21


In [53]:
1248 / 95672

0.01304456894389163